# Country-specific support schemes

This worksheet summarizes the different country regulations and support schemes relatives to PV self-consumption, with or without battery. The output is large matrix saved in an excel file providing values for each country. 

For the sake of openness, the selected format is the open-source Ipython Notebook format, which allows to easily follow and check all calculations and hypotheses. The input data is public data and is also made available through csv files. The original ipynb file can be downloaded and allows further reuse/modification/adaptation of the notebook: 

Sylvain Quoilin

Researcher at the European Commission (JRC).

E-mail: sylvain.quoilin@ec.europa.eu - Twitter: @squoilin

NB: The views expressed are purely those of the author and may not in any circumstances be regarded as stating an official position of the European Commission.

January 2016


### Load libraries

In [1]:
import pandas as pd
import numpy as np

#http://www.res-legal.eu/search-by-country/
#http://www.iea.org/policiesandmeasures/renewableenergy/?country=Denmark

### General assumptions and data

In [2]:
ratio_wholesale = 0.9      # ratio of the wholesale price to which electricity can be sold to the grid
Nyears = 20                # Project lifetime, years
rate_USD_EUR = 1.3285      # Exchange rate between EUR and USD
AdditionalRate = 0.03      # Additional rate on to top of the bonds to obtain WACC

datafolder = 'data/'

# Load general data:
retail_data = pd.read_excel(datafolder + 'Eurostat - per component household consumption - nrg_pc_204_c.xls')
wholesale_data = pd.read_excel(datafolder + 'wholesale.xlsx',index_col=0)
bonds_data = pd.read_excel(datafolder + 'governmental bonds.xlsx',index_col=0,skiprows=range(8),skip_footer=3)
PV_data = pd.read_excel(datafolder + 'PVCapacityFactors.xlsx',index_col=0,skiprows=range(4))


countries = ['Test','Flanders','Wallonia','France','UK','Spain','Portugal','Denmark','Hungary','Romania','Germany','Italy']
fields = ['WACC','bonds_10years','P_retail','P_retail_grid','P_retail_energy','P_retail_TL','P_wholesale','P_support_SC','C_grid_SC','C_grid_fixed','C_grid_kW','C_TL_SC','P_FtG','C_grid_FtG','C_TL_FtG','support_INV','support_kW','net_metering']

# Define the general dataframe where all country characteristics will be saved:
data = pd.DataFrame(np.nan,index=countries,columns=fields).fillna(0)

# function that gets the data from different sources in places it in the main matrix:
def get_data(retail_data,wholesale_data,country,code):
    
    # Tariff (source: Eurostat):
    retail_temp = retail_data[(retail_data.geo == code) & (retail_data.consom == 4161903) & (retail_data.currency == 'EUR')]
    retail_temp.index = retail_temp['breakdown']
    data.P_retail_grid[country] = retail_temp['2014S2 ']['DP_NC'] * 1000       # Grid fees in the retail tariff, €/MWh
    data.P_retail_TL[country] = retail_temp['2014S2 ']['DP_TL'] * 1000         # Tax and levies in the retail tariff, €/MWh
    data.P_retail_energy[country] = retail_temp['2014S2 ']['DP_ES'] * 1000     # Energy component of the retail tariff, €/MWh
    data.P_retail[country] = data.P_retail_grid[country] + data.P_retail_TL[country] + data.P_retail_energy[country]
    
    data.P_wholesale[country] = wholesale_data.P_wholesale[code]*1000         # Wholesale electricity price, €/MWh    
    data.bonds_10years[country] = bonds_data['2014'][code]/100



# Generic test case

In [3]:
country = 'Test'

data['P_retail_grid'][country] = 68.40          # Grid fees in the retail tariff, €/MWh
data['P_retail_TL'][country] = 60               # Tax and levies in the ret  ail tariff, €/MWh
data['P_retail_energy'][country] = 60           # Energy component of the retail tariff, €/MWh
data['P_retail'][country] = 188.40              # Retail price of electricity, €/MWh
data['P_wholesale'][country] = 52.6             # Wholesale electricity price, €/MWh    
data['bonds_10years'][country] = 0.013          # Rate of governmental bonds, -
data['WACC'][country] = 0.0226                  # weighted average cost of capital, - 
data['P_FtG'][country] = 52.641                 # Purchase price of electricity fed to the grid, €/MWh
data['P_support_SC'][country]  = 0              # Support to self-consumption, €/MWh
data['C_grid_SC'][country]  = 0                 # Grid fees for self-consumed electricity, €/MWh
data['C_TL_SC'][country] = 0                    # Tax and levies for self-consumed electricity, €/MWh
data['C_grid_fixed'][country]  = 0              # Fixed fees for connexion, €
data['C_grid_kW'][country]  = 0                 # Grid fees proportional to maximum connection capacity, €/kW
data['C_grid_FtG'][country] = 0                 # Grid fees for electricity fed to the grid, €/MWh
data['C_TL_FtG'][country] = 0                   # Tax and levies for electricity fed to the grid, €/MWh
data['support_INV'][country] = 0                # Investment support, % of investment
data['support_kW'][country] = 0                 # Investment support, per kWp
data['net_metering'][country] = False           # Boolean value stating if the net metering scheme is used

# Germany

In [4]:
country = 'Germany'
code = 'DE'

'''
Amount of feed-in:
https://de.wikipedia.org/wiki/Erneuerbare-Energien-Gesetz

Other source for the retail price structure:
Strompreis für Haushalte - BDEW
https://www.bdew.de/internet.nsf/id/9D1CF269C1282487C1257E22002BC8DD/$file/150409%20BDEW%20zum%20Strompreis%20der%20Haushalte%20Anhang.pdf
For 2015, various versions!!   No major differences though

'''

get_data(retail_data,wholesale_data,country,code)

data['WACC'][country] = data.bonds_10years[country] + AdditionalRate
data['P_FtG'][country] = 127                # Purchase price of electricity fed to the grid, €/MWh
data['P_support_SC'][country]  = 0              # Support to self-consumption, €/MWh
data['C_grid_SC'][country]  = 0                 # Grid fees for self-consumed electricity, €/MWh
data['C_TL_SC'][country] = 0                    # Tax and levies for self-consumed electricity, €/MWh
data['C_grid_fixed'][country]  = 0              # Fixed fees for connexion, €
data['C_grid_kW'][country]  = 0                 # Grid fees proportional to maximum connection capacity, €/kW
data['C_grid_FtG'][country] = 0                 # Grid fees for electricity fed to the grid, €/MWh
data['C_TL_FtG'][country] = 0                   # Tax and levies for electricity fed to the grid, €/MWh
data['support_INV'][country] = 0                # Investment support, % of investment
data['support_kW'][country] = 0                 # Investment support, per kWp
data['net_metering'][country] = False           # Boolean value stating if the net metering scheme is used

# Belgium - Wallonia

In [5]:
country = 'Wallonia'
code = 'BE'

get_data(retail_data,wholesale_data,country,code)

# Qualiwatt:
#http://www.cwape.be/docs/?doc=2479
Nyears_GC = 5             # Garanteed period, years
#amount = 190.87                # 5-years subsidy for january 2016, €/kWp/year, obtained by averaging over all DSOs
amount = 222.8574 + 17.43096

# Cost calculations:
i = data['WACC'][country] = data.bonds_10years[country] + AdditionalRate # Weighted average cost of capital: 3% added on top of the bonds
CRF = i * (1+i)**Nyears/((1+i)**Nyears-1)  # Capital Recovery Factor, %
CRF_GC = i * (1+i)**Nyears_GC/((1+i)**Nyears_GC-1)

data['P_support_SC'][country]  = 0            # Support to self-consumption, €/MWh
data['C_grid_SC'][country]  = 0   # Grid fees for self-consumed electricity, €/MWh
data['C_TL_SC'][country] = 0    # Tax and levies for self-consumed electricity, €/MWh
data['C_grid_fixed'][country]  = 0   # Fixed fees for connexion, €
data['C_grid_kW'][country]  = 0   # Grid fees proportional to maximum connection capacity, €/kW
     
data['support_INV'][country]
data['P_FtG'][country] = data.P_wholesale[country]*ratio_wholesale   # Purchase price of electricity fed to the grid, €/MWh
data['C_grid_FtG'][country] = 0             # Grid fees for electricity fed to the grid, €/MWh
data['C_TL_FtG'][country] = 0               # Tax and levies for electricity fed to the grid, €/MWh

data['support_INV'][country] = 0            # Investment support, % of investment
data['support_kW'][country] = amount/CRF_GC            # Investment support, per kWp

# Net metering is expected to stop in 2018 (however the subsidy still takes it into account for 2 years)
data['net_metering'][country] = False 


# Belgium - Flanders

In [6]:
country = 'Flanders'
code = 'BE'

get_data(retail_data,wholesale_data,country,code)

# Green certificates:
#http://www.vlaanderen.be/nl/bouwen-wonen-en-energie/elektriciteit-aardgas-en-verwarming/groenestroomcertificaten-voor-zonnepanelen
#http://www.vreg.be/nl/installaties-dienst-genomen-vanaf-1-juli-2015
P_GC = 93               # Price of green certificates, €/MWh
Nyears_GC = 15             # Garanteed period for GC, years
BF_GC = 0                  # Banding factor (number of GC allocated per MWh), none since july 2015

# Cost calculations:
i = data['WACC'][country] = data.bonds_10years[country] + AdditionalRate # Weighted average cost of capital: 3% added on top of the bonds
CRF = i * (1+i)**Nyears/((1+i)**Nyears-1)  # Capital Recovery Factor, %
CRF_GC = i * (1+i)**Nyears_GC/((1+i)**Nyears_GC-1)

P_support_PV = P_GC*BF_GC*CRF/CRF_GC  # Support for PV, independently of Self-consumption, €/MWh
data['P_support_SC'][country]  = P_support_PV            # Support to self-consumption, €/MWh
data['P_FtG'][country] = data.P_wholesale[country]*ratio_wholesale + P_support_PV    # Purchase price of electricity fed to the grid, €/MWh
data['net_metering'][country] = True 
# Since 1.7.2015, there is a prosumer tariff for the grid per kW of installed PV capacity:
# http://www.vreg.be/nl/prosumententarief-1#sect2.1
data['C_grid_kW'][country] = 89.03


# Denmark

In [7]:
# Hourly net metering with feed-in-tariff
#http://www.iea.org/policiesandmeasures/pams/denmark/name-42926-en.php?s=dHlwZT1yZSZzdGF0dXM9T2s,&return=PGRpdiBjbGFzcz0ic3ViTWVudSI-PGRpdiBjbGFzcz0iYnJlYWRjcnVtYnMiPjxhIGhyZWY9Ii8iPkludGVybmF0aW9uYWwgRW5lcmd5IEFnZW5jeSZ6d25qOzwvYT4mbmJzcDsmZ3Q7Jm5ic3A7PGEgaHJlZj0iL3BvbGljaWVzYW5kbWVhc3VyZXMvIj5Qb2xpY2llcyBhbmQgTWVhc3VyZXM8L2E-Jm5ic3A7Jmd0OzxhIGhyZWY9Ii9wb2xpY2llc2FuZG1lYXN1cmVzL3JlbmV3YWJsZWVuZXJneS9pbmRleC5waHAiPiZuYnNwO1JlbmV3YWJsZSBFbmVyZ3k8L2E-Jm5ic3A7Jmd0OyZuYnNwO1NlYXJjaCBSZXN1bHQ8L2Rpdj4,

country = 'Denmark'
code = 'DK'

get_data(retail_data,wholesale_data,country,code)

#Feed-in-tariff
FIT = 80
FIT= 37.4

# Cost calculations:
i = data['WACC'][country] = data.bonds_10years[country] + AdditionalRate # Weighted average cost of capital: 3% added on top of the bonds
CRF = i * (1+i)**Nyears/((1+i)**Nyears-1)  # Capital Recovery Factor, %
CRF_GC = i * (1+i)**Nyears_GC/((1+i)**Nyears_GC-1)

data['P_support_SC'][country]  = FIT            # Support to self-consumption, €/MWh
data['C_grid_SC'][country]  = 0   # Grid fees for self-consumed electricity, €/MWh
data['C_TL_SC'][country] = 0    # Tax and levies for self-consumed electricity, €/MWh
data['C_grid_fixed'][country]  = 0   # Fixed fees for connexion, €
data['C_grid_kW'][country]  = 0   # Grid fees proportional to maximum connection capacity, €/kW
     
data['support_INV'][country]
data['P_FtG'][country] = data.P_wholesale[country]*0.9 + FIT    # Purchase price of electricity fed to the grid, €/MWh
data['C_grid_FtG'][country] = 0             # Grid fees for electricity fed to the grid, €/MWh
data['C_TL_FtG'][country] = 0               # Tax and levies for electricity fed to the grid, €/MWh

data['support_INV'][country] = 0            # Investment support, % of investment
data['support_kW'][country] = 0            # Investment support, per kWp

data['net_metering'][country] = False 


# France

In [8]:
country = 'France'
code = 'FR'

get_data(retail_data,wholesale_data,country,code)

# Feed-in-tariff (December 2015):
#http://www.developpement-durable.gouv.fr/Quels-sont-les-tarifs-d-achats.html
# Considering a small PV system, not integrated into the building
FIT = 144      # Feed-in-tariff, €/MWh 

# Cost calculations:
i = data['WACC'][country] = data.bonds_10years[country] + AdditionalRate # Weighted average cost of capital: 3% added on top of the bonds
CRF = i * (1+i)**Nyears/((1+i)**Nyears-1)  # Capital Recovery Factor, %
CRF_GC = i * (1+i)**Nyears_GC/((1+i)**Nyears_GC-1)

data['P_support_SC'][country]  = FIT            # Support to self-consumption, €/MWh
data['P_FtG'][country] = FIT    # Purchase price of electricity fed to the grid, €/MWh
data['net_metering'][country] = False 


# Italy

In [9]:
# 6. Italy

#http://www.qualenergia.it/printpdf/articoli/20150121-italian-pv-market-ihs-economics-are-there-keeping-demand-700-mw-2015

#feed-in-tariff or net metering for small installations.
#Installations smaller than 3kW will be exempt from any charges
#Under Conto Energia (FIT), projects between 3 and 6kW will incur the largest cost, of €2.2/kW 
    # The system terminated in 2013. replaced by a tax credit
#For the Scambio sul posto (net-metering) scheme, installations of 3-20kW will pay a set annual fee of €30 
# http://www.qualenergia.it/articoli/20150505-scambio-sul-posto-pubblicate-le-nuove-regole-il-calcolo-del-contributo

country = 'Italy'
code = 'IT'

get_data(retail_data,wholesale_data,country,code)

# Cost calculations:
i = data['WACC'][country] = data.bonds_10years[country] + AdditionalRate # Weighted average cost of capital: 3% added on top of the bonds
CRF = i * (1+i)**Nyears/((1+i)**Nyears-1)  # Capital Recovery Factor, %
CRF_GC = i * (1+i)**Nyears_GC/((1+i)**Nyears_GC-1)

data['P_FtG'][country] = data.P_wholesale[country]*ratio_wholesale     # Purchase price of electricity fed to the grid, €/MWh

data['net_metering'][country] = True 



# Spain

In [10]:
# http://www.energiaysociedad.es/pdf/documentos/regulacion_tarifas/regulacion_nacional/EyS_RD_900_2015_autoconsumo.pdf
# https://www.boe.es/boe/dias/2015/10/10/pdfs/BOE-A-2015-10927.pdf - page 28
# http://tarifasgasluz.com/faq/tarifas/discriminacion-horaria
# We select the connexion type 2.0 DHA (< 10 kW, < 1 kV, 2 time-of-day periods)
# Selected period: 1st January 2016

country = 'Spain'
code = 'ES'

get_data(retail_data,wholesale_data,country,code)

# Cost calculations:
i = data['WACC'][country] = data.bonds_10years[country] + AdditionalRate # Weighted average cost of capital: 3% added on top of the bonds
CRF = i * (1+i)**Nyears/((1+i)**Nyears-1)  # Capital Recovery Factor, %
CRF_GC = i * (1+i)**Nyears_GC/((1+i)**Nyears_GC-1)

data['C_grid_SC'][country]  = 49.033   # Grid fees for self-consumed electricity, €/MWh
data['C_grid_kW'][country]  = 8.989169   # Grid fees proportional to maximum connection capacity, €/kW
# the power is defined as the maximum recorded household demand. If there is battery, it is the maximum
# exchange with the grid, plus the maximum power generation by the PV system. ???????
data['P_support_SC'][country]  = 0            # Support to self-consumption, €/MWh
data['P_FtG'][country] = 0    # Purchase price of electricity fed to the grid, €/MWh
data['net_metering'][country] = False 



# UK

In [11]:
country = 'UK'
code = 'UK'

get_data(retail_data,wholesale_data,country,code)

# Feed-in-tariff (January 2016):
#https://www.ofgem.gov.uk/publications-and-updates/feed-tariff-scheme-tariff-table-1-january-2016-pv-only
# Taking the middle rate for a 4 kW system:
convert = 1.42
FIT = 10.83 * 10 * convert      # Feed-in-tariff, €/MWh 

#Electricity fed into the grid is called exported electricity, and receives a small additional export tariff, currently (June 2013) 4.5p per KWh
export_premium = 47.7*convert

# Cost calculations:
i = data['WACC'][country] = data.bonds_10years[country] + AdditionalRate # Weighted average cost of capital: 3% added on top of the bonds
CRF = i * (1+i)**Nyears/((1+i)**Nyears-1)  # Capital Recovery Factor, %
CRF_GC = i * (1+i)**Nyears_GC/((1+i)**Nyears_GC-1)

data['P_support_SC'][country]  = FIT         # Support to self-consumption, €/MWh
data['support_INV'][country]
data['P_FtG'][country] = export_premium + FIT    # Purchase price of electricity fed to the grid, €/MWh
data['net_metering'][country] = False 



## Save data to excel

In [12]:
data.to_pickle('outputs/regulations.pickle')
data.to_excel('outputs/regulations.xls')